# Import dfs

In [176]:
import requests
import json
import pandas as pd

df_main = pd.read_csv('../data/cleaned_summary.csv', index_col = [0]).reset_index(drop = True)
df_geo = pd.read_csv('../data/address_coords.csv')
df_main.head()

,posted,date,time,city,state,shape,duration,summary,images,year,Keyword
0,03/11/06,1973-01-08,21:00:00,Muscle Shoals,AL,Oval,20 seconds,Glowing orange detailess object with outline s...,NaN,1973,"['orange', 'object', 'moving', 'sky']"
1,11/20/02,1973-01-12,03:00:00,"Farmington (SE of, deserted area, Hwy 44)",NM,Light,aprox:2-3 min,A large bright light apeared seemingly from no...,NaN,1973,"['large', 'bright', 'light']"
2,02/14/08,1973-01-14,19:00:00,Scranton,PA,Rectangle,10 seconds,Rectangular object moving at a very high rate ...,NaN,1973,"['object', 'moving']"
3,01/05/16,1973-02-10,02:00:00,Moundville,AL,Cigar,90 seconds,Cigar-shaped low flying after heavy snow storm.,NaN,1973,['flying']
4,10/30/06,1973-03-01,06:00:00,Winston-Salem,NC,Light,15 sec.,Saw a fast moving light that perfomed an insta...,NaN,1973,"['moving', 'light']"


In [177]:
df_geo.head()

,Address,Latitude,Longitude
0,"Newark, DE",39.683748,-75.749642
1,"Columbus, OH",39.961195,-82.998805
2,"Franklin, ME",44.587511,-68.231624
3,"East Greenwich, RI",41.634267,-71.513032
4,"Mentor Headlands, OH",41.739739,-81.311756


In [178]:
# combine city and state

df_main['Address'] = df_main['city'] + ', ' + df_main['state']

# drop city and state columns

#df_main.drop(['city', 'state'], axis = 1, inplace = True)

In [179]:
df_geo.head()

,Address,Latitude,Longitude
0,"Newark, DE",39.683748,-75.749642
1,"Columbus, OH",39.961195,-82.998805
2,"Franklin, ME",44.587511,-68.231624
3,"East Greenwich, RI",41.634267,-71.513032
4,"Mentor Headlands, OH",41.739739,-81.311756


In [180]:
top_states = df_main['state'].value_counts().nlargest(10)

print(top_states)
print(sum(top_states))

CA    9504
FL    4785
WA    4051
NY    3487
TX    3472
PA    3065
AZ    2784
OH    2582
IL    2519
NC    2211
Name: state, dtype: int64
38460


In [164]:
# get value counts of cities and select top 10
top_cities = df_main['Address'].value_counts().nlargest(10)

print(top_cities)
print(sum(top_cities))

Phoenix, AZ        416
Seattle, WA        402
Las Vegas, NV      349
Portland, OR       323
San Diego, CA      316
Los Angeles, CA    308
Tucson, AZ         269
Orlando, FL        242
Albuquerque, NM    236
Chicago, IL        234
Name: Address, dtype: int64
3095


In [165]:
# get 1979-2019 data for top 10 cities
## filter df to only include top 10 cities
df_top_cities = df_main[df_main['Address'].isin(top_cities.index)]

## filter df to only include 1979-2019
df_top_cities = df_top_cities[df_top_cities['year'].isin(range(1979, 2020))]
df_top_cities.head()

## combine date and time to unix timestamp
# df_top_cities['date'] = pd.to_datetime(df_top_cities['date'])
# df_top_cities['time'] = pd.to_datetime(df_top_cities['time'])
# df_top_cities['timestamp'] = df_top_cities['date'] + df_top_cities['time']
# df_top_cities['date'] = pd.to_datetime(df_top_cities['date'])
# df_top_cities['time'] = pd.to_datetime(df_top_cities['time'], format='%H:%M:%S')

# Concatenate the date and time columns into a single column
df_top_cities['datetime'] = pd.to_datetime(df_top_cities['date'] + ' ' + df_top_cities['time'])

# Convert the datetime objects to Unix timestamps
df_top_cities['unix_timestamp'] = df_top_cities['datetime'].apply(lambda x: int(x.timestamp()))


In [166]:
# merge with geo data

df_top_cities = df_top_cities.merge(df_geo, on = 'Address', how = 'left')
df_top_cities.head()

,posted,date,time,shape,duration,summary,images,year,Keyword,Address,datetime,unix_timestamp,Latitude,Longitude
0,08/22/14,1979-06-23,00:00:00,Unknown,unknown,"Loss of time, and white to blue cloud.",NaN,1979,['white'],"Las Vegas, NV",1979-06-23 00:00:00,298944000,36.171573,-115.139123
1,05/15/06,1979-08-20,21:00:00,Disk,5 minutes,Mother and two children recall ufo experience,NaN,1979,"['two', 'ufo']","Phoenix, AZ",1979-08-20 21:00:00,304030800,33.448379,-112.074018
2,02/01/07,1979-12-08,20:30:00,Circle,10-minutes,The night when I saw the disk UFO!,NaN,1979,['night'],"Albuquerque, NM",1979-12-08 20:30:00,313533000,35.084413,-106.650428
3,07/05/08,1980-05-05,04:00:00,Light,5 seconds,Los Angeles CA. 28 years ago. Spheric light tr...,NaN,1980,['light'],"Los Angeles, CA",1980-05-05 04:00:00,326347200,34.052221,-118.243706
4,02/01/07,1981-12-01,21:00:00,Light,15 minutes,17 lights that appeared they were playing tag ...,NaN,1981,['night'],"Orlando, FL",1981-12-01 21:00:00,376088400,28.538372,-81.378941


In [167]:
# Select the 'Latitude' and 'Longitude' columns from the dataframe
lat_lon_df = df_top_cities[['Address', 'Latitude', 'Longitude', 'year']]

# Drop duplicate rows to get unique combinations of latitude and longitude
unique_lat_lon_df = lat_lon_df.drop_duplicates()

# Optionally, you can reset the index of the resulting dataframe
unique_lat_lon_df = unique_lat_lon_df.reset_index(drop=True)

unique_lat_lon_df.rename({'Address': 'Custom ID'}, axis=1, inplace=True)

In [168]:
unique_lat_lon_df.to_csv('../data/unique_lat_lon.csv')

In [169]:
unique_lat_lon_df

,Custom ID,Latitude,Longitude,year
0,"Las Vegas, NV",36.171573,-115.139123,1979
1,"Phoenix, AZ",33.448379,-112.074018,1979
2,"Albuquerque, NM",35.084413,-106.650428,1979
3,"Los Angeles, CA",34.052221,-118.243706,1980
4,"Orlando, FL",28.538372,-81.378941,1981
...,...,...,...,...
292,"Los Angeles, CA",34.052221,-118.243706,2019
293,"Seattle, WA",47.606197,-122.332076,2019
294,"Phoenix, AZ",33.448379,-112.074018,2019
295,"Albuquerque, NM",35.084413,-106.650428,2019


In [37]:
unique_years = unique_lat_lon_df.groupby('Custom ID')['year'].unique()
unique_years_df = unique_years.reset_index()
unique_years_df.to_csv('../data/unique_years_top_10_cities.csv')

In [44]:
unique_years_df.iloc[0:9, 1][0]

array([1979, 1981, 1989, 1992, 1993, 1994, 1995, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [29]:
year_count_Vegas = df_top_cities.loc[df_top_cities['Address']=='Las Vegas, NV']['year'].value_counts()

#print(top_cities)
#print(sum(top_cities))

In [30]:
year_count_Vegas

2015    32
2012    29
2014    24
2007    20
2016    16
2010    16
2002    16
2019    14
2018    13
2017    12
2001    12
2006    11
2013    11
2011    11
2005     9
2000     9
2004     8
2003     8
2008     8
1999     8
2009     7
1998     6
1997     5
1995     4
1992     2
1991     2
1982     1
1994     1
1987     1
1979     1
Name: year, dtype: int64

# Pheonix Data Gathering 1979 to 2005

In [196]:
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

#add the access token you got from NOAA
Token = 'vxUAXWnUSMTKcRGtnOmJHDdlKVwyKzag'

#Phoenix Sky Harbor International Airport station ID
station_id = 'GHCND:USW00023183'
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1979-2006 ... (token limit reached -> 5 per second)
for year in range(1979, 2006):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023183&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_pheonix_79_05 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_pheonix_79_05['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_pheonix_79_05[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]


working on year 1979
working on year 1980
working on year 1981
working on year 1982
working on year 1983
working on year 1984
working on year 1985
working on year 1986
working on year 1987
working on year 1988
working on year 1989
working on year 1990
working on year 1991
working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005


# Pheonix Data Gathering 2006 to 2010

In [199]:
#Phoenix Sky Harbor International Airport station ID
station_id = 'GHCND:USW00023183'
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2006-2010 ... (token limit reached -> 5 per second)
for year in range(2006, 2011):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023183&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_pheonix_06_10 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_pheonix_06_10['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_pheonix_06_10[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2006
working on year 2007
working on year 2008
working on year 2009
working on year 2010


# Pheonix Data Gathering 2011 to 2015

In [202]:
#Phoenix Sky Harbor International Airport station ID
station_id = 'GHCND:USW00023183'
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2011-2015 ... (token limit reached -> 5 per second)
for year in range(2011, 2016):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023183&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_pheonix_11_15 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_pheonix_11_15['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_pheonix_11_15[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2011
working on year 2012
working on year 2013
working on year 2014
working on year 2015


# Pheonix Data Gathering 2016 to 2019

In [204]:
#Phoenix Sky Harbor International Airport station ID
station_id = 'GHCND:USW00023183'
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2016-2019 ... (token limit reached -> 5 per second)
for year in range(2016, 2020):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023183&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_pheonix_16_19 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_pheonix_16_19['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_pheonix_16_19[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2016
working on year 2017
working on year 2018
working on year 2019


# Pheonix Data Gathering 2020 to 2022

In [205]:
#Phoenix Sky Harbor International Airport station ID
station_id = 'GHCND:USW00023183'
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2020-2022 ... (token limit reached -> 5 per second)
for year in range(2020, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023183&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_pheonix_20_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_pheonix_20_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_pheonix_20_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2020
working on year 2021
working on year 2022


# Concatenate all Phoenix, AZ, dataframes

In [206]:
# concatenate all Phoenix, AZ, dataframes
df_pheonix = pd.concat([df_pheonix_79_05, df_pheonix_06_10, df_pheonix_11_15, df_pheonix_16_20, df_pheonix_20_22], ignore_index=True)

In [208]:
df_pheonix.to_csv('../data/Weather/df_pheonix_api.csv', index=False)

# Albuquerque Data Gathering 1992 to 1996

In [209]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1992-1996 ... (token limit reached -> 5 per second)
for year in range(1992, 1997):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_92_96 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_92_96['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_92_96[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996


# Albuquerque Data Gathering 1997 to 2001

In [210]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1997-2001 ... (token limit reached -> 5 per second)
for year in range(1997, 2002):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_97_01 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_97_01['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_97_01[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001


# Albuquerque Data Gathering 2002 to 2006

In [211]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2002-2006 ... (token limit reached -> 5 per second)
for year in range(2002, 2007):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_02_06 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_02_06['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_02_06[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006


# Albuquerque Data Gathering 2007 to 2011

In [212]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2007-2011 ... (token limit reached -> 5 per second)
for year in range(2007, 2012):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_07_11 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_07_11['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_07_11[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2007
working on year 2008
working on year 2009
working on year 2010
working on year 2011


# Albuquerque Data Gathering 2012 to 2016

In [213]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2012-2016 ... (token limit reached -> 5 per second)
for year in range(2012, 2017):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_12_16 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_12_16['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_12_16[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2012
working on year 2013
working on year 2014
working on year 2015
working on year 2016


# Albuquerque Data Gathering 2017 to 2019

In [214]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2017-2019 ... (token limit reached -> 5 per second)
for year in range(2017, 2020):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_17_19 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_17_19['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_17_19[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2017
working on year 2018
working on year 2019


# Albuquerque Data Gathering 2020 to 2022

In [215]:
# Albuquerque International Sunport station ID
station_id = 'GHCND:USW00023050'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2020-2022 ... (token limit reached -> 5 per second)
for year in range(2020, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023050&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_albuquerque_20_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_albuquerque_20_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_albuquerque_20_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2020
working on year 2021
working on year 2022


# Concatenate all Albuquerque, NM, dataframes

In [216]:
# concatenate all Albuquerque, NM, dataframes
df_albuquerque = pd.concat([df_albuquerque_92_96, df_albuquerque_97_01, df_albuquerque_02_06, df_albuquerque_07_11, df_albuquerque_12_16, df_albuquerque_17_19, df_albuquerque_20_22], ignore_index=True)
df_albuquerque.to_csv('../data/Weather/df_albuquerque_api.csv', index=False)

# Los Angeles Data Gathering 1980 to 1984

In [217]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1980-1984 ... (token limit reached -> 5 per second)
for year in range(1980, 1985):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_80_84 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_80_84['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_80_84[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1980
working on year 1981
working on year 1982
working on year 1983
working on year 1984


# Los Angeles Data Gathering 1986 to 1990 (ufo dataset doesnt contain 1985)

In [218]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1986-1990 ... (token limit reached -> 5 per second)
for year in range(1986, 1991):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_86_90 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_86_90['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_86_90[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1986
working on year 1987
working on year 1988
working on year 1989
working on year 1990


# Los Angeles Data Gathering 1992 to 1996 (ufo dataset doesnt contain 1991)

In [220]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1992-1996 ... (token limit reached -> 5 per second)
for year in range(1992, 1997):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_92_96 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_92_96['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_92_96[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996


# Los Angeles Data Gathering 1997 to 2003

In [221]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1997-2003 ... (token limit reached -> 5 per second)
for year in range(1997, 2004):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_97_03 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_97_03['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_97_03[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003


# Los Angeles Data Gathering 2004 to 2008

In [222]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2004-2008 ... (token limit reached -> 5 per second)
for year in range(2004, 2009):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_04_08 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_04_08['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_04_08[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008


# Los Angeles Data Gathering 2009 to 2013

In [223]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2009-2013 ... (token limit reached -> 5 per second)
for year in range(2009, 2014):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_09_13 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_09_13['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_09_13[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013


# Los Angeles Data Gathering 2014 to 2018

In [224]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2014-2018 ... (token limit reached -> 5 per second)
for year in range(2014, 2019):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_14_18 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_14_18['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_14_18[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2014
working on year 2015
working on year 2016
working on year 2017
working on year 2018


# Los Angeles Data Gathering 2019 to 2022

In [225]:
# Los Angeles LAX station ID
station_id = 'GHCND:USW00023174'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2019-2022... (token limit reached -> 5 per second)
for year in range(2019, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023174&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_losangeles_19_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_losangeles_19_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_losangeles_19_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2019
working on year 2020
working on year 2021
working on year 2022


# Concatenate all Los Angeles, CA, dataframes

In [226]:
# concatenate all Los Angeles, CA, dataframes
df_losangeles = pd.concat([df_losangeles_80_84, df_losangeles_86_90, df_losangeles_92_96, df_losangeles_97_03, df_losangeles_04_08, df_losangeles_09_13, df_losangeles_14_18, df_losangeles_19_22], ignore_index=True)
df_losangeles.to_csv('../data/Weather/df_losangeles_api.csv', index=False)

# Portland, OR, Data Gathering 1983 to 1985

In [227]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1983-1985 ... (token limit reached -> 5 per second)
for year in range(1983, 1986):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_83_85 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_83_85['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_83_85[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1983
working on year 1984
working on year 1985


# Portland, OR, Data Gathering 1991 to 1994 ('86 to '90 not in UFO)

In [228]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1991-1994 ... (token limit reached -> 5 per second)
for year in range(1991, 1995):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_91_94 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_91_94['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_91_94[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1991
working on year 1992
working on year 1993
working on year 1994


# Portland, OR, Data Gathering 1996 to 2000 (1995 not in UFO)

In [229]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1996-2000 ... (token limit reached -> 5 per second)
for year in range(1996, 2001):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_96_00 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_96_00['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_96_00[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000


# Portland, OR, Data Gathering 2001 to 2005

In [230]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2001-2005 ... (token limit reached -> 5 per second)
for year in range(2001, 2006):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_01_05 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_01_05['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_01_05[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005


# Portland, OR, Data Gathering 2006 to 2010

In [231]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2006-2010 ... (token limit reached -> 5 per second)
for year in range(2006, 2011):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_06_10 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_06_10['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_06_10[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2006
working on year 2007
working on year 2008
working on year 2009
working on year 2010


# Portland, OR, Data Gathering 2011 to 2015

In [232]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2011-2015 ... (token limit reached -> 5 per second)
for year in range(2011, 2016):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_11_15 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_11_15['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_11_15[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2011
working on year 2012
working on year 2013
working on year 2014
working on year 2015


# Portland, OR, Data Gathering 2016 to 2019

In [233]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2016-2019 ... (token limit reached -> 5 per second)
for year in range(2016, 2020):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_16_19 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_16_19['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_16_19[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2016
working on year 2017
working on year 2018
working on year 2019


# Portland, OR, Data Gathering 2020 to 2022

In [234]:
# Portland, OR, Intl Airport station ID
station_id = 'GHCND:USW00024229'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2020-2022 ... (token limit reached -> 5 per second)
for year in range(2020, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024229&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_portland_20_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_portland_20_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_portland_20_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2020
working on year 2021
working on year 2022


# Concatenate all Portland, OR, dataframes

In [235]:
# concatenate all Portland, OR, dataframes
df_portland = pd.concat([df_portland_83_85, df_portland_91_94, df_portland_96_00, df_portland_01_05, df_portland_06_10, df_portland_11_15, df_portland_16_19, df_portland_20_22], ignore_index=True)
df_portland.to_csv('../data/Weather/df_portland_api.csv', index=False)

# San Diego, CA, Data Gathering 1983 to 1985

In [237]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1983-1985 ... (token limit reached -> 5 per second)
for year in range(1983, 1986):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_83_85 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_83_85['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_83_85[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1983
working on year 1984
working on year 1985


# San Diego, CA, Data Gathering 1989 to 1993

In [238]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1989-1993 ... (token limit reached -> 5 per second)
for year in range(1989, 1994):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_89_93 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_89_93['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_89_93[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1989
working on year 1990
working on year 1991
working on year 1992
working on year 1993


# San Diego, CA, Data Gathering 1994 to 1998

In [239]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1994-1998 ... (token limit reached -> 5 per second)
for year in range(1994, 1999):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_94_98 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_94_98['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_94_98[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998


# San Diego, CA, Data Gathering 1999 to 2003

In [240]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1999-2003 ... (token limit reached -> 5 per second)
for year in range(1999, 2004):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_99_03 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_99_03['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_99_03[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003


# San Diego, CA, Data Gathering 2004 to 2008

In [241]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2004-2008 ... (token limit reached -> 5 per second)
for year in range(2004, 2009):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_04_08 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_04_08['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_04_08[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008


# San Diego, CA, Data Gathering 2009 to 2013

In [242]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2009-2013 ... (token limit reached -> 5 per second)
for year in range(2009, 2014):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_09_13 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_09_13['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_09_13[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013


# San Diego, CA, Data Gathering 2014 to 2018

In [243]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2014-2018 ... (token limit reached -> 5 per second)
for year in range(2014, 2019):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_14_18 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_14_18['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_14_18[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2014
working on year 2015
working on year 2016
working on year 2017
working on year 2018


# San Diego, CA, Data Gathering 2019 to 2022

In [244]:
# San Diego, CA, Intl Airport station ID
station_id = 'GHCND:USW00023188'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2019-2022 ... (token limit reached -> 5 per second)
for year in range(2019, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023188&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_sandiego_19_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_sandiego_19_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_sandiego_19_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2019
working on year 2020
working on year 2021
working on year 2022


# Concatenate all San Diego, CA, dataframes

In [245]:
# concatenate all San Diego, CA, dataframes
df_sandiego = pd.concat([df_sandiego_83_85, df_sandiego_89_93, df_sandiego_94_98, df_sandiego_99_03, df_sandiego_04_08, df_sandiego_09_13, df_sandiego_14_18, df_sandiego_19_22], ignore_index=True)
df_sandiego.to_csv('../data/Weather/df_sandiego_api.csv', index=False)

# Tucson, AZ, Data Gathering 1988-1992

In [248]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1988-1992 ... (token limit reached -> 5 per second)
for year in range(1988, 1993):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_88_92 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_88_92['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_88_92[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1988
working on year 1989
working on year 1990
working on year 1991
working on year 1992


# Tucson, AZ, Data Gathering 1993-1997

In [249]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1993-1997 ... (token limit reached -> 5 per second)
for year in range(1993, 1998):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_93_97 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_93_97['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_93_97[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997


# Tucson, AZ, Data Gathering 1998-2002

In [251]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1998-2002 ... (token limit reached -> 5 per second)
for year in range(1998, 2003):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_98_02 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_98_02['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_98_02[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002


# Tucson, AZ, Data Gathering 2003-2007

In [252]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2003-2007 ... (token limit reached -> 5 per second)
for year in range(2003, 2008):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_03_07 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_03_07['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_03_07[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2003
working on year 2004
working on year 2005
working on year 2006
working on year 2007


# Tucson, AZ, Data Gathering 2008-2012

In [253]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2008-2012 ... (token limit reached -> 5 per second)
for year in range(2008, 2013):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_08_12 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_08_12['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_08_12[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2008
working on year 2009
working on year 2010
working on year 2011
working on year 2012


# Tucson, AZ, Data Gathering 2013-2017

In [254]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2013-2017 ... (token limit reached -> 5 per second)
for year in range(2013, 2018):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_13_17 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_13_17['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_13_17[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017


# Tucson, AZ, Data Gathering 2018-2022

In [255]:
# Tucson, AZ, Intl Airport station ID
station_id = 'GHCND:USW00023160'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2018-2022 ... (token limit reached -> 5 per second)
for year in range(2018, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00023160&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_tucson_18_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_tucson_18_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_tucson_18_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2018
working on year 2019
working on year 2020
working on year 2021
working on year 2022


# Concatenate all Tucson, AZ, dataframes

In [256]:
# concatenate all Tucson, AZ, dataframes
df_tucson = pd.concat([df_tucson_88_92, df_tucson_93_97, df_tucson_98_02, df_tucson_03_07, df_tucson_08_12, df_tucson_13_17, df_tucson_18_22], ignore_index=True)
df_tucson.to_csv('../data/Weather/df_tucson_api.csv', index=False)

# Seattle, WA, Data Gathering 1982-1987

In [257]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1982-1987 ... (token limit reached -> 5 per second)
for year in range(1982, 1988):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_82_87 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_82_87['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_82_87[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1982
working on year 1983
working on year 1984
working on year 1985
working on year 1986
working on year 1987


# Seattle, WA, Data Gathering 1992-1997

In [258]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1992-1997 ... (token limit reached -> 5 per second)
for year in range(1992, 1998):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_92_97 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_92_97['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_92_97[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997


# Seattle, WA, Data Gathering 1998-2003

In [259]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1998-2003 ... (token limit reached -> 5 per second)
for year in range(1998, 2004):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_98_03 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_98_03['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_98_03[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003


# Seattle, WA, Data Gathering 2004-2008

In [260]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2004-2008 ... (token limit reached -> 5 per second)
for year in range(2004, 2009):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_04_08 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_04_08['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_04_08[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008


# Seattle, WA, Data Gathering 2009-2013

In [261]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2009-2013 ... (token limit reached -> 5 per second)
for year in range(2009, 2014):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_09_13 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_09_13['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_09_13[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013


# Seattle, WA, Data Gathering 2014-2017

In [262]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2014-2017 ... (token limit reached -> 5 per second)
for year in range(2014, 2018):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_14_17 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_14_17['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_14_17[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2014
working on year 2015
working on year 2016
working on year 2017


# Seattle, WA, Data Gathering 2018-2022

In [263]:
# Seattle Tacoma Intl Airport station ID
station_id = 'GHCND:USW00024233'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2018-2022 ... (token limit reached -> 5 per second)
for year in range(2018, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00024233&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_seattle_18_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_seattle_18_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_seattle_18_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2018
working on year 2019
working on year 2020
working on year 2021
working on year 2022


# Concatenate all Seattle, WA, dataframes

In [264]:
# concatenate all Seattle, WA, dataframes
df_seattle = pd.concat([df_seattle_82_87, df_seattle_92_97, df_seattle_98_03, df_seattle_04_08, df_seattle_09_13, df_seattle_14_17, df_seattle_18_22], ignore_index=True)
df_seattle.to_csv('../data/Weather/df_seattle_api.csv', index=False)

# Orlando, FL, Data Gathering 1981-1985

In [269]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1981-1985 ... (token limit reached -> 5 per second)
for year in range(1981, 1986):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_81_85 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_81_85['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_81_85[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1981
working on year 1982
working on year 1983
working on year 1984
working on year 1985


# Orlando, FL, Data Gathering 1995-2000

In [270]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 1995-2000 ... (token limit reached -> 5 per second)
for year in range(1995, 2001):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_95_00 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_95_00['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_95_00[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000


# Orlando, FL, Data Gathering 2001-2006

In [271]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2001-2006 ... (token limit reached -> 5 per second)
for year in range(2001, 2007):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_01_06 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_01_06['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_01_06[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006


# Orlando, FL, Data Gathering 2007-2012

In [273]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2007-2012 ... (token limit reached -> 5 per second)
for year in range(2007, 2013):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_07_12 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_07_12['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_07_12[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2007
working on year 2008
working on year 2009
working on year 2010
working on year 2011
working on year 2012


# Orlando, FL, Data Gathering 2013-2017

In [279]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2013-2017 ... (token limit reached -> 5 per second)
for year in range(2013, 2018):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_13_17 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_13_17['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_13_17[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2013
working on year 2014
working on year 2015
working on year 2016
working on year 2017


# Orlando, FL, Data Gathering 2018-2022

In [280]:
# Orlando Intl Airport station ID
station_id = 'GHCND:USW00012815'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2018-2022 ... (token limit reached -> 5 per second)
for year in range(2018, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00012815&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_orlando_18_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_orlando_18_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_orlando_18_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

working on year 2018
working on year 2019
working on year 2020
working on year 2021
working on year 2022


# Concatenate all Orlando, FL, dataframes

In [281]:
# concatenate all Orlando, FL, dataframes
df_orlando = pd.concat([df_orlando_81_85, df_orlando_95_00, df_orlando_01_06, df_orlando_07_12, df_orlando_13_17, df_orlando_18_22], ignore_index=True)
df_orlando.to_csv('../data/Weather/df_orlando_api.csv', index=False)

# Las Vegas, NV, Data Gathering 2020-2022

In [285]:
# North Las Vegas North Airport station ID
station_id = 'GHCND:USW00053123'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2020-2022 ... (token limit reached -> 5 per second)
for year in range(2020, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00053123&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_vegas_20_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_vegas_20_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_vegas_20_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

df_vegas_20_22.to_csv('../data/Weather/df_vegas_api.csv', index=False)

working on year 2020
working on year 2021
working on year 2022


# Chicago, IL, Data Gathering 2020-2022

In [286]:
# Chicago Ohare Airport station ID
station_id = 'GHCND:USW00094846'  
dates_temp = []
dates_prcp = []
temps = []
prcp = []

# for each year from 2020-2022 ... (token limit reached -> 5 per second)
for year in range(2020, 2023):
    year = str(year)
    print('working on year ' + year)

    # make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT12,WT13,WT14,WT15,WT16,WT17,WT18,WV01,WV03,WV07,WV18,WV20&limit=1000&stationid=GHCND:USW00094846&startdate=' + year + '-01-01&enddate=' + year + '-12-31', headers={'token':Token})
    # load the api response as a json
    d = json.loads(r.text)

    # get all items in the response which are weather descriptions (1,0)
    weather_types = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11', 'WT12', 'WT13','WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WV01','WV03','WV07','WV18','WV20']
    weather_data = {}
    for wt in weather_types:
        weather_data[wt] = [item for item in d['results'] if item['datatype'] == wt]

        # get the date field from all readings
        dates_temp += [item['date'] for item in weather_data[wt]]

# initialize dataframe
df_chicago_20_22 = pd.DataFrame()

# populate date and weather descriptions fields (cast string date to datetime and add other fields)
df_chicago_20_22['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
for wt in weather_types:
    df_chicago_20_22[wt] = [next((item['value'] for item in weather_data[wt] if item['date'] == d), None) for d in dates_temp]

df_chicago_20_22.to_csv('../data/Weather/df_chicago_api.csv', index=False)

working on year 2020
working on year 2021
working on year 2022
